# TP - Computação Natural
#### "Predict whether a mammogram mass is benign or malignant"

1. BI-RADS assessment: 1 to 5 (ordinal)  
2. Age: patient's age in years (integer)
3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
6. Severity: benign=0 or malignant=1 (binominal)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import stats

## Get the Data

In [ ]:
data = pd.read_csv('mammographic_masses.data.txt')
data

** Convert missing data (indicated by a ?) into NaN and add the appropriate column names (BI_RADS, age, shape, margin, density, and severity) **

In [ ]:
data = data.replace('?',np.nan)
data.columns = ['BI_RADS','Age','Shape','Margin','Density','Severity']
data

** Drop BI_RADS column because it has no influence on the severity forecast **

In [ ]:
data = data.drop(columns=['BI_RADS'])

** Convert datatype 'object' to 'float64' **  

In [ ]:
data.info()

In [ ]:
data = data.astype(float)
data

In [ ]:
data.info()

In [ ]:
data.describe()

### Analysing missing values
**First we get the missing values per feature.**

*Lets check them out as well*

In [ ]:
missing_values_feature = data.isnull().sum(axis=0)
graph = missing_values_feature.drop(labels='Severity')
graph

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.bar(graph.axes[0].to_list(), graph.values)

*We then develop a heatmap to give us some more information*

In [ ]:
sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

*Finally we can check the percentage of missing values per feature*

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
print(missing_value_df)

**After analysing the columns, we should have a look at the rows**

In [ ]:
data_missing = len(data.columns) - (data.apply(lambda x: x.count(), axis=1))
missing_values_data_rows = pd.DataFrame({'data_missing':data_missing})
missing_values_data_rows.sort_values('data_missing',inplace=True,ascending=False)
missing_values_data_rows

**Now lets analyse the missing data per class (Severity = 0 or Severity = 1).**

*First we group the missing values per class*

In [ ]:
grouped_data = data.groupby('Severity')
missing_values_class = grouped_data.count().rsub(grouped_data.size(), axis=0)
missing_values_class

*Now we split the dataframe per class so we can draw our plot*

In [ ]:
m_new_1, m_new_2 = missing_values_class.head(1), missing_values_class.tail(1)

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.bar(m_new_1.axes[1].to_list(), m_new_1.values[0], label = "Severity 0")
plt.subplot(131)
plt.bar(m_new_2.axes[1].to_list(), m_new_2.values[0], label = "Severity 1")
plt.legend()

**Finally, for each class we're going to calculate the number of rows that have 1 and 2 NaN values**

In [ ]:
rows_mv1_sv0 = 0
rows_mv2_sv0 = 0
rows_mv1_sv1 = 0
rows_mv2_sv1 = 0

In [ ]:
for index, row in data.iterrows():
    if(row['Severity'] == 0):
        if(row.isnull().sum() == 1):
            rows_mv1_sv0 += 1
        elif(row.isnull().sum() == 2):
            rows_mv2_sv0 += 1
    else:
        if(row.isnull().sum() == 1):
            rows_mv1_sv1 += 1
        elif(row.isnull().sum() == 2):
            rows_mv2_sv1 += 1

*We create a dataframe only for visualization purpose*

In [ ]:
numberofnan_class = pd.DataFrame(np.array([[rows_mv1_sv0,rows_mv2_sv0], [rows_mv1_sv1,rows_mv2_sv1]]), 
                                    index=['Severity 0','Severity 1'], columns=['1 NaN', '2 NaN'])
numberofnan_class

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.bar(['1 NaN', '2 NaN'], [rows_mv1_sv0,rows_mv2_sv0], label = "Severity 0")
plt.subplot(131)
plt.bar(['1 NaN', '2 NaN'], [rows_mv1_sv1,rows_mv2_sv1], label = "Severity 1")
plt.legend()

*With this information we can also see the number of rows with 1 or 2 missing values per class*

In [ ]:
nan_class = pd.DataFrame(np.array([[rows_mv1_sv0+rows_mv2_sv0], [rows_mv1_sv1+rows_mv2_sv1]]), 
                                    index=['Severity 0','Severity 1'], columns=['Sum'])
nan_class

In [ ]:
plt.figure(figsize=(3, 5))
plt.bar(['Severity 0','Severity 1'],[rows_mv1_sv0+rows_mv2_sv0,rows_mv1_sv1+rows_mv2_sv1])

** The missing data seems randomly distributed, so we decided to go with the following strategy: **

* Drop rows with 2 NaN values

* Replace the NaN values from rows with 1 missing value

*First we get the mode of every feature for each class*

In [ ]:
mode_sv0 = data[data['Severity'] == 0].mode()
mode_sv1 = data[data['Severity'] == 1].mode()
mode_sv0 = mode_sv0.drop([1])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(mode_sv0)
    print(mode_sv1)

*After we create conditions to replace the NaN values on rows with 1 missing value.*

*For that we need the index of the rows which have 1 missing value.*

In [ ]:
rows_1nan = missing_values_data_rows.index[missing_values_data_rows['data_missing'] == 1].tolist()
mask_sv0 = (data['Severity'] == 0) & (data.index.isin(rows_1nan))
mask_sv1 = (data['Severity'] == 1) & (data.index.isin(rows_1nan))

*We can now proceed and replace the missing values for their class mode* 

In [ ]:
data.loc[mask_sv0, 'Shape'] = data.loc[mask_sv0, 'Shape'].fillna(mode_sv0.loc[0,'Shape'])
data.loc[mask_sv0, 'Margin'] = data.loc[mask_sv0, 'Margin'].fillna(mode_sv0.loc[0,'Margin'])
data.loc[mask_sv0, 'Density'] = data.loc[mask_sv0, 'Density'].fillna(mode_sv0.loc[0,'Density'])
data.loc[mask_sv1, 'Age'] = data.loc[mask_sv1, 'Age'].fillna(mode_sv1.loc[0,'Age'])
data.loc[mask_sv1, 'Shape'] = data.loc[mask_sv1, 'Shape'].fillna(mode_sv1.loc[0,'Shape'])
data.loc[mask_sv1, 'Margin'] = data.loc[mask_sv1, 'Margin'].fillna(mode_sv1.loc[0,'Margin'])
data.loc[mask_sv1, 'Density'] = data.loc[mask_sv1, 'Density'].fillna(mode_sv1.loc[0,'Density'])
data

*Finally, we can drop rows with NaN values because the only ones that are left are the ones with 2 NaN*

In [ ]:
data = data.dropna()
data.index = np.arange(1, len(data) + 1)
data

** !!! (Ou podemos meter os dados que faltavam, ver isto) !!! **

In [ ]:
data.describe()

## Exploratory Data Analysis

** Countplot of the Severity (Benign 0 vs Malignant 1) **

In [ ]:
sns.countplot(x='Severity',data=data)

** Histogram showing Age based on the Severity column **

In [ ]:
sns.set_style('darkgrid')
g = sns.FacetGrid(data,hue="Severity",palette='coolwarm',size=6,aspect=2)
g = (g.map(plt.hist,'Age',bins=20,alpha=0.7)).add_legend()

##### Detect outliers: https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

### Detect Outliers using Box plot (Uni-variate outlier)

In [ ]:
sns.boxplot(x=data['Age'])

In [ ]:
sns.boxplot(x=data['Shape'])

In [ ]:
sns.boxplot(x=data['Margin'])

In [ ]:
sns.boxplot(x=data['Density'])

### Detect Outliers using Scatter plot (Multi-variate outlier)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(data['Age'], data['Shape'])
ax.set_xlabel('Age')
ax.set_ylabel('Shape')
#ax.set_ylabel('Margin')
#ax.set_ylabel('Density')
plt.show()

### Detect outliers using mathematical function Z-Score

In [ ]:
z = np.abs(stats.zscore(data))
threshold = 3
print(np.where(z > threshold))
# The first array contains the list of row numbers and second array respective column numbers

Column 3 (density) has all outliers

### Detect outliers using IQR Score
Similar to Z-Score

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
iqr = Q3 - Q1
print(iqr)

In [ ]:
# Não curti ...
print(data < (Q1 - 1.5 * iqr)) |(data > (Q3 + 1.5 * iqr))

### Remove Outliers using Z-Score

##### + explanations: https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

In [ ]:
# Só fazer 1 vez
data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
data.index = np.arange(1, len(data) + 1)
data

### Converting pandas dataframes to numpy arrays

In [ ]:
X_train = data.drop('Severity',axis=1).to_numpy()
y_train = data['Severity'].to_numpy()

### Normalizing the attribute data using StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)

## Neural Networks